# Indicator synthesis

In [ ]:
%run ../notebook_preamble.ipy

import seaborn as sn
import pandas_datareader
import altair as alt
from altair_saver import save
from eis.utils.data_processing import *
from selenium import webdriver
from dotenv import load_dotenv, find_dotenv
import os

In [ ]:
#Altair saving code
w = make_altair_save()

save_dir = f"{project_dir}/reports/figures/final_report"

def save_altair_(f,n):
    save_altair(f,n,w,fig_path=save_dir)

### Load metadata

In [ ]:
with open(f"{data_path}/aux/eu_codes_names.txt",'r') as infile:
    eu_27_other_codes = infile.read().split(', ')
    
eu_codes_lookup = {x.split(':')[0].strip().lower(): x.split(':')[1].strip() for x in eu_27_other_codes}
eu_codes_lookup['gr']= 'Greece'
eu_codes_lookup['gb'] = 'United Kingdom'
eu_codes_lookup['uk'] = 'United Kingdom'
eu_codes_lookup['is'] = 'Iceland'
eu_codes_lookup['il'] = 'Israel'
eu_codes_lookup['ua'] = 'Ukraine'

eu_codes = set(eu_codes_lookup.keys())

In [ ]:
# country_codes = pd.read_csv(
#     'https://www.eea.europa.eu/data-and-maps/data/waterbase-lakes-4/country-codes-and-abbreviations-32-records/country-codes-and-abbreviations-32-records/at_download/file')

# eu_codes = set([x.lower() for x in country_codes['ISO2']]+['uk',''])

In [ ]:
# with open(f"{project_dir}/data/aux/eu_country_codes.txt",'r') as infile:
#     eu_codes_2 = infile.read().split(', ')

In [ ]:
with open(f"{project_dir}/data/aux/all_clean_names.json",'r') as infile:
    
    clean_names= json.load(infile)
    
with open(f"{project_dir}/data/aux/eurostat_clean_names.json",'r') as infile:
    
    es_clean_names= json.load(infile)

In [ ]:
all_names = {**clean_names,**es_clean_names}

In [ ]:
#Lookup to change a couple of country codes
country_name_changes = {'gb':'uk','gr':'el'}

In [ ]:
# eu_code_name_lookup = {r['ISO2'].lower():r['Country name'] for rid,r in country_codes.iterrows()}
# eu_code_name_lookup['uk'] = 'United Kingdom'
# eu_code_name_lookup['el'] = 'Greece'
# eu_code_name_lookup['cz']

## Load data

In [ ]:
#Create a unified indicator table
inds = pd.concat([pd.read_csv(f"{project_dir}/data/processed/{name}_indicators.csv") for name in
                 ['official','web']])

inds['country'] = [x.lower() for x in inds['country']]

inds['variable_clean'] = [all_names[x] for x in inds['variable']]

inds['country'] = [country_name_changes[x] if x in country_name_changes.keys() else x for x in inds['country']]

### Additional eurostat data for normalisation

In [ ]:
#We downloaded this with the make_eurostat script function and processed in 4b scaling factors
pop = pd.read_csv(f"{project_dir}/data/processed/eurostat_demo_table.csv")

#Lower names
pop['country'] = pop['country'].apply(lambda x: x.lower())

In [ ]:
#Normalise

norm_lookup = {'educ_uoe_grad02':'pop_student_age',
               'hrst_st_nsec2':'pop_working_age',
               'github_users':'pop_working_age',
               'python_downloads':'pop_working_age',
               'tech_meetups':'pop_working_age'}

new_cont = []

for x in set(inds['variable']):
    
    #If we don't need to normalise
    if x not in norm_lookup.keys():
        
        #Add to the new container
        new_cont.append(inds.loc[inds['variable']==x].reset_index(drop=True))
        
    else: #If we need to normalise
        
        rel = inds.loc[inds['variable']==x]
        
        #Filter population
        #Variable to normalise with
        var = norm_lookup[x]
        
        #Filter population to focus on it
        rel_pop = pop.loc[pop['variable']==var]
        
        
        #Merge with population
        merg = pd.merge(rel,rel_pop,left_on=['country','year'],right_on=['country','year'],how='left')
        
        #Normalise
        merg['value'] = merg['value']/merg['demo_pjan']
        
        #Add a norm append
        merg['variable'] = x+'_norm'
        
        #focus on our variables
        out = merg[['country','year','variable','value','variable_clean']]

        new_cont.append(out)
        
ind_norm = pd.concat(new_cont)

In [ ]:
#Focus on the EU countries
ind_norm_eu = ind_norm.loc[[x in eu_codes for x in ind_norm['country']]].reset_index(drop=True)

ind_norm_eu['country_full'] = ind_norm_eu['country'].map(eu_codes_lookup)

In [ ]:
#Add normalised names to lookup
ind_norm_eu['variable_clean'] = [row['variable_clean'] if 'norm' not in row['variable'] else 
                                 row['variable_clean'] + " (per capita)" for rid,row in ind_norm_eu.iterrows()]

for rid,row in ind_norm_eu.drop_duplicates('variable').iterrows():
    
    if row['variable'] not in all_names.keys():
        
        all_names[row['variable']] = row['variable_clean']


In [ ]:
ind_norm_eu_rec = ind_norm_eu.loc[(ind_norm_eu['year']>=2015)&((ind_norm_eu['year']<=2019))]

In [ ]:
#We want to addd missing values for those variables without data in a year

missing_dfs = []

for x in set(ind_norm_eu_rec['variable_clean']):
    
    #Get all data in the year
    rel = ind_norm_eu_rec.loc[ind_norm_eu_rec['variable_clean']==x]
    
    #What years are there?
    years = set(rel['year'])
    
    #What are the missing years?
    missing = set(np.arange(2015,2020))-years
    
    #For each year missing we will create a df with missing values
    for x in missing:
        
        year_df = rel.loc[rel['year']==list(years)[0]]
        year_df['year'] = x
        year_df['value'] = np.nan
        
        missing_dfs.append(year_df)

ind_norm_eu_rec_missing = pd.concat([ind_norm_eu_rec,pd.concat(missing_dfs)])

In [ ]:
def coverage_chart(df,_vars):
    #Missing years and missing countries
    
    #Focus on selected variables
    df_ = df.loc[df['variable'].isin(_vars)]
    
    #Count missing values by country and year
    miss_c = df_.groupby(['variable_clean','country_full'])['value'].apply(
        lambda x: np.sum(x.isna()==True)).reset_index(drop=False)
    miss_y = df_.groupby(['variable_clean','year'])['value'].apply(
        lambda x: np.sum(x.isna()==True)).reset_index(drop=False)

    #Create altair objects with variable positions
    y_pos = alt.Y('variable_clean',sort=alt.EncodingSortField('value','mean',order='descending'))
    x_pos = alt.X('country_full:O',sort=alt.EncodingSortField('value','sum',order='descending'))
    x_pos_y = alt.Y('year:O')
    
    #Country chart
    c = alt.Chart(miss_c).mark_point(
        filled=True,stroke='black',strokeWidth=1).encode(
        y=y_pos,x=x_pos,size='value',color='value').properties(width=450,height=250)

    #Year chart
    y = alt.Chart(miss_y).mark_point(
        filled=True,stroke='black',strokeWidth=1).encode(y=alt.Y('variable_clean',axis=alt.Axis(labels=False),
                                                                title=''),
                                                                 x=x_pos_y,size='value',
                                                         color=alt.Color('value',title=['Number of missing', 'values']))
    #Concatenate variables
    sh = alt.hconcat(c,y).resolve_scale(y='shared')

    return sh
    

In [ ]:
cov = coverage_chart(ind_norm_eu_rec_missing,set(ind_norm_eu_rec['variable']))

save_altair_(cov,'fig_17_missing')

cov

In [ ]:
#Calculate correlations between variables
ind_2017 = ind_norm_eu.loc[ind_norm_eu['year']==2018]

vs = ind_2017.pivot_table(index='country_full',columns='variable',values='value').corr()

vs.index.name = 'v1'
vs.columns.name = 'v2'

vs_2 = vs.reset_index(
    drop=False).melt(id_vars='v1')

for v in ['v1','v2']:
    vs_2[v+'_clean'] = [all_names[x] for x in vs_2[v]]

In [ ]:
vs_2['corr_label'] = [str(np.round(x,2)) for x in vs_2['value']]

In [ ]:
x_pos = alt.X('v1_clean:N',sort=alt.EncodingSortField('value','mean',order='descending'),title=None)
y_pos = alt.Y('v2_clean:N',sort=alt.EncodingSortField('value','mean',order='descending'),title=None)

base = alt.Chart(vs_2).mark_rect().encode(x=x_pos,y=y_pos)

text = base.mark_text().encode(
    text='corr_label',
    color=alt.condition(
        alt.datum.value > 0.5, 
        alt.value('white'),
        alt.value('black')
    )
)

col = base.encode(color='value')

f = (col+text).properties(width=500,height=300)

save_altair_(f,'fig_18_correlation')

f

#### Country position

In [ ]:
def calculate_zscore(series):
    
    return (series - series.mean())/(series.std())

In [ ]:
ind_2018 = ind_norm_eu.query('year == 2017').pivot_table(index='country_full',
                                                        columns='variable_clean',
                                                         values='value').apply(
    lambda x: calculate_zscore(x)).reset_index(drop=False)

#Remove countries with lots of missing values
missing_values = ind_2018.set_index('country_full').isna().sum(axis=1)

low_missing_countries = missing_values.index[missing_values<5]

In [ ]:
ind_2018_long = ind_2018.loc[ind_2018['country_full'].isin(low_missing_countries)].melt(id_vars='country_full')

In [ ]:
rank = (alt
        .Chart(ind_2018_long)
        .mark_rect()
        .encode(
        y=alt.Y('country_full',title=None,sort=alt.EncodingSortField('value',op='median',order='descending')),
        x = alt.X('variable_clean',title='Indicator',sort=alt.EncodingSortField('value',op='mean',order='descending')),
        color=alt.Color('value',title='Z-score',scale=alt.Scale(scheme='purpleorange')))).properties(height=450)

save_altair_(rank,'fig_19_ranking')

rank

### Analyse correlation between indicators and EIS

#### Read EIS data

In [ ]:
import re

In [ ]:
file = "https://ec.europa.eu/docsroom/documents/36062/attachments/1/translations/en/renditions/native"
file_2 = "https://ec.europa.eu/docsroom/documents/41864/attachments/1/translations/en/renditions/native"

eis_df = pd.read_excel(file_2,sheet_name=3)

In [ ]:
#Create a cleaner version with nicer columns etc
eis_ = eis_df.copy()

eis_.columns = [c+'_'+str(int(x)) if pd.isnull(x)==False else c for c,x in zip(eis_.columns,eis_.iloc[0])]
eis_.columns = [re.sub(" ","_",x.lower()) for x in eis_.columns]

eis_ = eis_.iloc[2:].reset_index(drop=True).rename(columns={'unnamed:_0':'country_name',
                                                           'unnamed:_1':'country_code'})

eis_['country_code'] = eis_['country_code'].apply(lambda x: x.lower())



In [ ]:
eis_2019 = eis_.iloc[:,:3]

#eis_2018['year'] = 2018

eis_2019.rename(columns={'summary_innovation_index_2019':'eis_2019'},inplace=True)

eis_2019['eis_2019'] = pd.to_numeric(eis_2019['eis_2019'])

In [ ]:
#Allocate countries to quartiles (ranking)

eis_2019['position'] = pd.qcut(eis_2019['eis_2019'],q=np.arange(0,1.1,0.2),labels=False)

#### Combine with our data

In [ ]:
#We focus on our variables of interest
ind_eis = pd.merge(ind_norm_eu,eis_2019,left_on=['country'],right_on=['country_code'],
                   how='inner')

In [ ]:
#For our variables we want to keep similar years

vars_2018 = ['hrst_st_nsec2_norm','educ_uoe_grad02_norm','tech_meetups_norm','github_users_norm']

year_var = {v:2019 if v not in vars_2018 else 2018 for v in set(ind_eis['variable'])}

In [ ]:
ind_eis['year_match'] = [row['year']==year_var[row['variable']] for _id,row in ind_eis.iterrows()]

ind_eis_sh = ind_eis.loc[ind_eis['year_match']==True].reset_index(drop=True)

### Coverage

In [ ]:
#What variables do we keep?
key_vars = ['hrst_st_nsec2_norm','educ_uoe_grad02_norm','isoc_sk_dskl_i','github_users_norm','tech_meetups_norm']

In [ ]:
# cov = coverage_chart(ind_norm_eu_rec_missing,key_vars)

# save_altair_(cov,"selected_coverage")

# cov

#### Correlation matrix

In [ ]:
def make_correlation_chart(df,indicators):
    '''Creates a barchart with selected indicators
    
    '''
    
    eis_corr = df.pivot_table(index='country',columns='variable',values='value')[indicators].corrwith(
    eis_2019.set_index('country_code')['eis_2019']).reset_index(name='correlation')

    eis_corr['variable_clean'] = eis_corr['variable'].map(all_names)

    corr_ch = alt.Chart(eis_corr).mark_bar().encode(
        y=alt.Y('variable_clean',
                title='Variable',sort=alt.EncodingSortField('correlation',order='descending')),
        x=alt.X('correlation',title=['Correlation coefficient','with EIS-2019'])).properties(height=150,width=200)

    #corr_ch.save(f"{fig_path}/correlation_chart.png",webdriver=driver,method='selenium',scale_factor=3)
    return(corr_ch)
    
    

In [ ]:
corr = make_correlation_chart(ind_eis_sh,key_vars).properties(width=300,height=100)

save_altair_(corr,'fig_20_correlation_w_eis')

corr

#### Interactive chart

In [ ]:
# def make_eis_link_chart(df,variable):
    
#     #Subset the df to focus on the variable of interest
#     df_sub = df.loc[df['variable']==variable].reset_index(drop=False).dropna()
    
#     #This is a lookup to relabel the outputs of our model
#     ind_label_lookup = df_sub.to_dict(orient='index')

#     #We want to calculate the residuals of the model that regresses eis on the variable of interest
#     mod = sm.OLS(df_sub['value'],sm.add_constant(df_sub['eis_2018']),missing='drop')
#     outp = mod.fit()
    
#     #Extract the residuals and sort them
#     outp = outp.resid.sort_values().reset_index(name='residual')
#     outp['country'] = [ind_label_lookup[x]['country_full'] for x in outp['index']]

#     #Merge residuals table with main table. We want them in the same table to link variables in the visualisation below
#     df_sub_2 = df_sub.merge(outp[['country_full','residual']],left_on='country_full',right_on='country_full')
    
#     #Plotting
#     #Create selection object
#     sel = alt.selection_interval()
    
#     #Create selection object
#     col = alt.condition(sel,'position:N',alt.value('lightgrey'))
    
#     #Create the scatter
#     scatter = (alt.Chart(df_sub_2)
#                .mark_point(filled=True,size=40)
#                .encode(y='eis_2018',x='value',tooltip=['country_full'],
#                        color=col))
    
#     #Create the regression line
#     reg_line = scatter.transform_regression('value','eis_2018',method='linear').mark_line(strokeDash=[2,2])
    
#     #Create the scatter + line
#     scatter_line = ((scatter + reg_line)
#                     .properties(height=150,width=150,title=all_names[variable][:30]+'...')
#                     .add_selection(sel))

#     #Add a barchart with residuals
#     res_bar = (alt.Chart(df_sub_2)
#                .mark_bar()
#                .encode(
#                    y=alt.Y('country_full',sort=alt.SortByEncoding('x',order='descending'),axis=alt.Axis(grid=True)),
#                    x='residual',
#                    color=col,
#                    tooltip=['country_full'])).properties(height=400,width=150).add_selection(sel)

#     comp = alt.vconcat(scatter_line,res_bar)
#     return([df_sub_2,comp])

In [ ]:
def make_eis_link_chart_2(df,variable):
    '''Version of the eis_link chart but visualising ranking changes instead of residuals
    
    '''
    
    #Subset the df to focus on the variable of interest
    df_sub = df.loc[df['variable']==variable].reset_index(drop=False).dropna()
    
    #This is a lookup to relabel the outputs of our model
    ind_label_lookup = df_sub.to_dict(orient='index')
    
    rank_changes = df_sub.set_index('country_full').assign(
            ranking_difference = lambda x: x['value'].rank()-x['eis_2019'].rank())[
            'ranking_difference'].reset_index(drop=False)

    #Merge residuals table with main table. We want them in the same table to link variables in the visualisation below
    df_sub_2 = df_sub.merge(rank_changes[['country_full','ranking_difference']],left_on='country_full',
                            right_on='country_full')
    
    #Plotting
    #Create selection object
    sel = alt.selection_interval()
    
    #Create selection object
    col = alt.condition(sel,alt.Color('position:N',title='Quintile'),
                                      alt.value('lightgrey'))
    
    col_2 = alt.condition(sel,alt.Color('position:N',legend=None),
                                      alt.value('lightgrey'))
    
    
    #Create the scatter
    scatter = (alt.Chart(df_sub_2)
               .mark_point(filled=True,size=40)
               .encode(y=alt.Y('eis_2019',title='EIS 2019'),
                       x='value',tooltip=['country'],
                       color=col_2))
    
    #Create the regression line
    reg_line = scatter.transform_regression('value','eis_2019',method='linear').mark_line(
        strokeDash=[2,2])
    
    #Create the scatter + line
    scatter_line = ((scatter + reg_line)
                    .properties(height=150,width=150,title=all_names[variable][:30]+'...')
                    .add_selection(sel)).resolve_scale(color='independent')

    #Add a barchart with residuals
    res_bar = (alt.Chart(df_sub_2)
               .mark_bar()
               .encode(
                   y=alt.Y('country_full',title='Country',
                           sort=alt.SortByEncoding('x',order='descending'),axis=alt.Axis(grid=True)),
                   x=alt.X('ranking_difference',title=['Change in ranking','EIS 2019 - indicator']),
                   color=col,
                   tooltip=['country_full'])).properties(height=400,width=150).add_selection(sel)

    comp = alt.vconcat(scatter_line,res_bar)
    return([df_sub_2,comp])

In [ ]:
def make_multiple_link_charts(df,variables):
    '''Creates concatenated charts 
    
    '''
    outs = [make_eis_link_chart_2(df,v) for v in variables]

    scatter_charts = alt.hconcat(*[x[1] for x in outs]).resolve_scale(x='shared')

    return scatter_charts
    

In [ ]:
scatter_charts = make_multiple_link_charts(ind_eis_sh,key_vars)
save_altair_(scatter_charts,"scatters")

scatter_charts.save(f"{save_dir}/indicator_eis_comparison.html")
save_altair_(scatter_charts,'fig_21_scatters')

scatter_charts

#### Ranking change

In [ ]:
def rank_change_plot(df,indicators):
    '''Creates a plot about country ranking differences according to various indicators
    
    '''

    rank_changes = df.groupby('variable').apply(
        lambda x: x.set_index('country_full').assign(
            ranking_difference = lambda x: x['value'].rank()-x['eis_2019'].rank())[
            ['position','ranking_difference']]).reset_index(drop=False)
    
    #Create selection object
    sel = alt.selection_interval(encodings=['x'])
    
    #Create color object
    col = alt.condition(sel,alt.Color('position:N',title='Quintile'),
                                      alt.value('lightgrey'))

    rank_changes_selected = rank_changes.loc[rank_changes['variable'].isin(indicators)]

    rank_changes_selected['variable_clean'] = rank_changes_selected['variable'].map(all_names)
    
    rank_chart = (alt.Chart(rank_changes_selected)
                  .mark_bar()
                  .encode(
                      x=alt.X('country_full',sort=alt.EncodingSortField('position',order='descending'),
                             title=None),
                      y=alt.Y('ranking_difference',title=['Ranking','difference']),
                      #color='position:N',
                      color=col,
                      tooltip=['country_full'],
                      row=alt.Row('variable_clean',header=alt.Header(labelOrient='top'),
                                  sort=[all_names[x] for x in key_vars]))
                  .properties(height=80,width=500).configure_axisX(grid=True)).add_selection(sel)
    
    return(rank_chart)
    

In [ ]:
rank_ch = rank_change_plot(ind_eis_sh,key_vars)

save_altair_(rank_ch,"ranking_change")

rank_ch.save(f"{fig_path}/indicator_rank_differences.html")
save_altair_(rank_ch,'fig_22_ranking_change')

rank_ch

### Alternatives

In [ ]:
key_vars_2 = ['isoc_ski_itemp','isoc_ci_ifp_iu']

In [ ]:
make_correlation_chart(ind_eis_sh,key_vars_2)

In [ ]:
make_multiple_link_charts(ind_eis_sh,key_vars_2)

In [ ]:
rank_change_plot(ind_eis_sh,key_vars_2)

### Some extra analysis for the internal workshop

In [ ]:
for x in key_vars:
    print(x)
    print(all_names[x])
    
    print('\n')

In [ ]:
for k in key_vars:
    print(all_names[k])